In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
import os

import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K

from typing import List
from tqdm import trange

import wandb
from wandb.keras import WandbCallback

import config, music_model, utils
from sklearn.utils import class_weight

TRAIN = False

### CONFIGURATION ###

wandb.login()

ROOT_PATH = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
MODEL_SAVE_PATH = os.path.join(ROOT_PATH, 'training', 'checkpoints', 'genre_classifier')
os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
MODEL_SAVE_PATH = os.path.join(MODEL_SAVE_PATH, 'model-{val_loss:.2f}.h5')
DATASET_NAME = 'lmd_matched_final_2048_cut'
USE_SMALL_GENRE_SET = DATASET_NAME == 'tf_data7dict'
USE_ONE_GPU = True

WEIGHTS_PATH = os.path.join(ROOT_PATH, 'training', 'checkpoints', 
                            'model_GPT_baseline_with_mse_vellmd_matched_2048', 
                            'model_GPT_baseline_with_mse_vellmd_matched_2048')

conf = config.Config("single_instruments_type", ROOT_PATH)

if USE_SMALL_GENRE_SET:
    conf.accepted_subgenres = ['folk', 'nes', 'maestro']
# If we need to use only the first GPU
if USE_ONE_GPU:
    conf.GPUS = tf.config.experimental.list_physical_devices('GPU')[0]
    conf.BATCH_SIZE = conf.BATCH_SIZE
    conf.GLOBAL_BATCH_SIZE = conf.BATCH_SIZE
    conf.num_devices = 1

### MODEL CREATION ###

if conf.num_devices > 1:
    print("Using multiple GPUs with Mirrored Strategy")
    with conf.training_strategy.scope():
        model = music_model.create_model(conf,
                                         num_genres=len(conf.accepted_subgenres),
                                         use_regularization=False,
                                         use_masking_layers=False)
else:
    print("Using single GPU/CPU device")
    model = music_model.create_model(conf,
                                     num_genres=len(conf.accepted_subgenres),
                                     use_regularization=False,
                                     use_masking_layers=False)

print("Loading pre-trained weights into the model and freezing weights...")
model.load_weights(WEIGHTS_PATH)
    
print("Collecting dataset splits...")
dataset_path = conf.dataset_paths[DATASET_NAME]
train_dataset, val_dataset, test_dataset = utils.get_dataset_splits(dataset_path, conf)

print("Modifying labels...")
train_dataset, val_dataset, test_dataset = \
    train_dataset.map(lambda x, y: (x, x[1])), \
    val_dataset.map(lambda x, y: (x, x[1])),   \
    test_dataset.map(lambda x, y: (x, x[1]))

2023-06-04 22:46:27.880633: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-04 22:46:28.025078: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-04 22:46:28.057841: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-04 22:46:28.682857: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

Using single GPU/CPU device


2023-06-04 22:46:32.105697: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-04 22:46:32.735464: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30487 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:86:00.0, compute capability: 7.0


Loading pre-trained weights into the model and freezing weights...
Modifying labels...


We extract the transformer's embeddings for the dataset and define the genre classifier on top of the model

In [3]:
music_transformer = tf.keras.Model(
    inputs=model.inputs, 
    outputs=model.get_layer('tfgpt2_model').output.last_hidden_state
)
music_transformer.trainable = False

class_gen = tf.keras.Sequential([
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(len(conf.accepted_subgenres), activation='relu'),
    tf.keras.layers.Softmax()
])

out_classes = class_gen(music_transformer.outputs[0])

genre_classifier = tf.keras.Model(inputs=music_transformer.inputs, outputs=out_classes)
genre_classifier.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), 
                         loss=tf.keras.losses.KLDivergence())

In [4]:
labels = np.concatenate([x[1] for x, y in train_dataset], axis=0)
genre_labels = np.where(labels > 0)[1]
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(genre_labels), y=genre_labels)
class_weights = {i: class_weights[i] for i in range(len(class_weights))}

In [5]:
if TRAIN:
    run = wandb.init(project="Music Generation", entity="marcello-e-federico",
                             group='genre_classification', job_type='train',
                             name='baseline_genre_class_full')

    genre_classifier.fit(x=train_dataset, validation_data=val_dataset, epochs=200, 
                         callbacks=[
                            tf.keras.callbacks.EarlyStopping(patience=30, restore_best_weights=True),
                            tf.keras.callbacks.ModelCheckpoint(filepath=MODEL_SAVE_PATH, 
                                save_best_only=True, save_weights_only=True),
                            tf.keras.callbacks.ReduceLROnPlateau(),
                            WandbCallback(save_model=False, 
                                          save_graph=False,log_weights=False)
                            ])

    run.finish()

In [6]:
genre_classifier.load_weights(os.path.join(os.path.dirname(MODEL_SAVE_PATH), 'model-0.47.h5'))

In [7]:
test_kld = genre_classifier.evaluate(x=test_dataset, batch_size=conf.BATCH_SIZE)

   1/1160 [..............................] - ETA: 1:51:33 - loss: 3.0867

2023-06-04 22:46:52.317790: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8400


1160/1160 [==============================] - 149s 124ms/step - loss: 0.3837


In [8]:
test_kld

0.3837237060070038

Also obtain the classification report

In [9]:
from tqdm import tqdm

gt_genre_vectors = np.concatenate(
    [y for _, y in tqdm(test_dataset)], 
    axis = 0
)

predicted_genre_vectors = np.concatenate(
    [genre_classifier(X) for X, _ in tqdm(test_dataset)],
    axis = 0
)

gt_genre_vectors.shape, predicted_genre_vectors.shape

100%|██████████| 1160/1160 [02:55<00:00,  6.62it/s]


((6955, 18), (6955, 18))

In [10]:
from utils import metrics_classification_report

metrics_classification_report(gt_genre_vectors, predicted_genre_vectors, conf)

6955it [00:00, 79138.03it/s]
/home/aborghesi/persistent/MusicGeneration/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/aborghesi/persistent/MusicGeneration/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'rock': {'precision': 0.5963407661520869,
  'recall': 0.3024064946361264,
  'f1-score': 0.40130819545979224,
  'support': 3449},
 'pop': {'precision': 0.574719800747198,
  'recall': 0.2731577389760284,
  'f1-score': 0.3703109327983952,
  'support': 3379},
 'dance': {'precision': 0.24427480916030533,
  'recall': 0.07872078720787208,
  'f1-score': 0.11906976744186046,
  'support': 813},
 'country': {'precision': 0.25510204081632654,
  'recall': 0.08833922261484099,
  'f1-score': 0.13123359580052493,
  'support': 849},
 'metal': {'precision': 0.10975609756097561,
  'recall': 0.031914893617021274,
  'f1-score': 0.04945054945054945,
  'support': 282},
 'classical': {'precision': 0.08356545961002786,
  'recall': 0.07211538461538461,
  'f1-score': 0.07741935483870968,
  'support': 416},
 'folk': {'precision': 0.17582417582417584,
  'recall': 0.023703703703703703,
  'f1-score': 0.041775456919060046,
  'support': 675},
 'blues': {'precision': 0.14189189189189189,
  'recall': 0.0446808510638297